In [2]:
import cv2
import os
import os.path as osp
import matplotlib.pyplot as plt
import numpy as np
import pickle
import pandas as pd
from skimage.morphology import skeletonize
from skimage.util import invert
from nebula.common import to_scale_one, write_pickle, read_pickle, display, display_color
from nebula.data.yg_ar.setup_data_image_hard import split_ar_anim_df

In [3]:
def get_loc_frame_from_data(data, loc=0.5):
    count = data.get(cv2.CAP_PROP_FRAME_COUNT)
    f_loc = int((count-1) * loc)
    
    data.set(cv2.CAP_PROP_POS_FRAMES, f_loc)
    ret, frame = data.read()
    
    return frame
    
    
def get_loc_frame(path, loc=0.5):
    data = cv2.VideoCapture(path)
    return get_loc_frame_from_data(data, loc=loc)


def get_n_frames(path, n=8):
    data = cv2.VideoCapture(path)
    res = [get_loc_frame_from_data(data, loc=i/n) for i in range(n)]
    
    return res


def get_locs_frames(path, locs=[0.15, 0.5, 0.85]):
    data = cv2.VideoCapture(path)
    res = [get_loc_frame_from_data(data, loc=loc) for loc in locs]
    return res
    

def to_gray_one(frame):
    return cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)


def to_gray(frames):
    res = [to_gray_one(f) for f in frames]
    return res


def crop_one(frame, top, bottom, left, right):
    return frame[top:-bottom, left:-right]


def crop(frames, top, bottom, left, right):
    res = [crop_one(f, top, bottom, left, right) for f in frames]
    return res


def save_frame(path, frame):
    cv2.imwrite(path, frame)
    

def read_gray_frame(path):
    return cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    
    
def to_jpg_name(file):
    return file.split(".")[0] + ".jpg"


def to_pickle_name(file):
    return file.split(".")[0] + ".pkl"


def get_files(dir, format=None):
    res = []
    for filename in os.listdir(dir):
        if format is None or filename.split(".")[-1] == format:
            res.append(filename)
    return res


def filename_to_labels(filename):
    
    filename = filename.split(".")[0]
    words = filename.split("_")
    
    orientation = words[-1]
    xangle = words[-2]
    yangle = words[-3]
    pants = words[-4]
    cloth = words[-6]
    hair = words[-8]
    action_type = words[-10]
    label = "_".join(words[:-10])
    
    return (
        orientation, 
        xangle, 
        yangle, 
        pants, 
        cloth, 
        hair, 
        action_type, 
        label
    )


def filename_to_labels_hard(filename):
    
    filename = filename.split(".")[0]
    words = filename.split("_")
    
    action = "_".join(words[:-12])
    action_type = words[-12]
    
    label_a = action
    label_at = "_".join([action, action_type])
    
    return (
        label_a,
        label_at,
        filename
    )


def read_pickle(dir):
    with open(dir, 'rb') as handle:
        b = pickle.load(handle)
    return b


def write_pickle(dir, data):
    with open(dir, 'wb') as handle:
        pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [4]:
def convert_middle_frame(
    base_dir,
    save_dir,
    overwrite=False,
    crop=dict(top=70, bottom=30, left=50, right=50)
):
    if not osp.exists(save_dir):
        os.makedirs(save_dir)
    
    files = get_files(base_dir)
    count = len(files)
    prev_progress = 0
    for i, file in enumerate(files):
        save_file = osp.join(save_dir, to_jpg_name(file))
        if osp.exists(save_file) and not overwrite:
            continue
        
        path = osp.join(base_dir, file)
        frame = to_gray_one(crop_one(get_loc_frame(path), **crop))
        
        save_frame(save_file, frame)
        
        cur_progress = int((i+1)*100/count)
        if cur_progress >= prev_progress + 2:
            print(f"progress: {cur_progress}%")
            prev_progress = cur_progress
            

def df_middle_frame(
    image_dir,
    save_path,
):
    files = get_files(image_dir)
    count = len(files)
    print(count)
    prev_progress = 0
    res = []
    for i, f in enumerate(files):
        frame = read_gray_frame(osp.join(image_dir, f))
        (
            label_a,
            label_ac,
            fname
        ) = filename_to_labels_hard(f)
        
        res.append(
            (
                frame, 
                label_a,
                label_ac,
                fname
            )
        )
        
        cur_progress = int((i+1)*100/count)
        if cur_progress >= prev_progress + 2:
            print(f"progress: {cur_progress}%")
            prev_progress = cur_progress

    df = pd.DataFrame(data=dict(zip(
        [
            "image", 
            "label_a",
            "label_at",
            "file_name"
        ], 
        np.transpose(res)
    )))
    
    df.to_pickle(save_path)
    
    return df

In [5]:
def convert_locs_frames(
    base_dir,
    save_dir,
    overwrite=False,
    crop=dict(top=70, bottom=30, left=50, right=50),
    locs=[0.15, 0.5, 0.85]
):
    if not osp.exists(save_dir):
        os.makedirs(save_dir)
    
    files = get_files(base_dir)
    count = len(files)
    prev_progress = 0
    for i, file in enumerate(files):
        save_file = osp.join(save_dir, to_pickle_name(file))
        if osp.exists(save_file) and not overwrite:
            continue
        
        path = osp.join(base_dir, file)
        frames = get_locs_frames(path, locs=locs)
        frames = [to_gray_one(crop_one(f, **crop)) for f in frames]
        
        write_pickle(save_file, frames)
        
        cur_progress = int((i+1)*100/count)
        if cur_progress >= prev_progress + 2:
            print(f"progress: {cur_progress}%")
            prev_progress = cur_progress
            

def df_locs_frames(
    pickle_dir,
    save_path,
):
    files = get_files(pickle_dir)
    count = len(files)
    print(count)
    prev_progress = 0
    res = []
    for i, f in enumerate(files):
        frames = read_pickle(osp.join(pickle_dir, f))
        (
            label_a,
            label_ac,
            fname
        ) = filename_to_labels_hard(f)
        
        res.append(
            (
                frames, 
                label_a,
                label_ac,
                fname
            )
        )
        
        cur_progress = int((i+1)*100/count)
        if cur_progress >= prev_progress + 2:
            print(f"progress: {cur_progress}%")
            prev_progress = cur_progress

    df = pd.DataFrame(data=dict(zip(
        [
            "images", 
            "label_a",
            "label_at",
            "file_name"
        ], 
        np.transpose(res)
    )))
    
    df.to_pickle(save_path)
    
    return df

In [6]:
from PIL import Image
import cv2 as cv
from scipy import ndimage
import random


def zoom(frame, x, y, z=100):
    img = Image.fromarray(frame)
    w, h = img.size
    z2 = z * 2/100.0
    img = img.crop((x - w / z2, y - h / z2, 
                    x + w / z2, y + h / z2))
    img = img.resize((w, h), Image.LANCZOS)
    return np.asarray(img)


def rotate(frame, degree):
    return ndimage.rotate(frame, degree, reshape=False)


def random_zoom(frame, min_loc=-10, max_loc=10, min_z=100, max_z=130):
    
    h, w = frame.shape
    
    x_offset = random.randint(min_loc, max_loc)
    y_offset = random.randint(min_loc, max_loc)
    z = random.randint(min_z, max_z)
    
    return zoom(frame, int(w/2)+x_offset, int(h/2)+y_offset, z)


def random_rotate(frame, min_d=-10, max_d=10):
    
    deg = random.randint(min_d, max_d)
    return rotate(frame, deg)
    
    

In [8]:
base_dir = 'C:/Users/aphri/Documents/t0002/pycharm/data/yg_ar/videos_hard'
save_dir = 'C:/Users/aphri/Documents/t0002/pycharm/data/yg_ar/v5f_hard'
v5f_pickle_path = 'C:/Users/aphri/Documents/t0002/pycharm/data/yg_ar/v5f_hard_df.pkl'

In [9]:
convert_locs_frames(
    base_dir=base_dir,
    save_dir=save_dir,
    overwrite=False,
    crop=dict(top=70, bottom=30, left=50, right=50),
    locs=[0.1, 0.3, 0.5, 0.7, 0.9]
)

progress: 2%
progress: 4%
progress: 6%
progress: 8%
progress: 10%
progress: 12%
progress: 14%
progress: 16%
progress: 18%
progress: 20%
progress: 22%
progress: 24%
progress: 26%
progress: 28%
progress: 30%
progress: 32%
progress: 34%
progress: 36%
progress: 38%
progress: 40%
progress: 42%
progress: 44%
progress: 46%
progress: 48%
progress: 50%
progress: 52%
progress: 54%
progress: 56%
progress: 58%
progress: 60%
progress: 62%
progress: 64%
progress: 66%
progress: 68%
progress: 70%
progress: 72%
progress: 74%
progress: 76%
progress: 78%
progress: 80%
progress: 82%
progress: 84%
progress: 86%
progress: 88%
progress: 90%
progress: 92%
progress: 94%
progress: 96%
progress: 98%
progress: 100%


In [10]:
df_locs_frames(
    pickle_dir=save_dir,
    save_path=v5f_pickle_path
)

20000
progress: 2%
progress: 4%
progress: 6%
progress: 8%
progress: 10%
progress: 12%
progress: 14%
progress: 16%
progress: 18%
progress: 20%
progress: 22%
progress: 24%
progress: 26%
progress: 28%
progress: 30%
progress: 32%
progress: 34%
progress: 36%
progress: 38%
progress: 40%
progress: 42%
progress: 44%
progress: 46%
progress: 48%
progress: 50%
progress: 52%
progress: 54%
progress: 56%
progress: 58%
progress: 60%
progress: 62%
progress: 64%
progress: 66%
progress: 68%
progress: 70%
progress: 72%
progress: 74%
progress: 76%
progress: 78%
progress: 80%
progress: 82%
progress: 84%
progress: 86%
progress: 88%
progress: 90%
progress: 92%
progress: 94%
progress: 96%
progress: 98%
progress: 100%


c:\users\aphri\miniconda3\lib\site-packages\numpy\core\fromnumeric.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = getattr(asarray(obj), method)(*args, **kwds)


,images,label_a,label_at,file_name
0,"[[[26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26,...",camel,camel_1,camel_1_hair_0_cloth_0_pants_0_Z1031_XON17_YON...
1,"[[[12, 12, 12, 11, 11, 12, 12, 12, 12, 12, 12,...",camel,camel_1,camel_1_hair_0_cloth_0_pants_0_Z1095_XOP17_YON...
2,"[[[44, 44, 44, 44, 44, 44, 43, 43, 45, 45, 45,...",camel,camel_1,camel_1_hair_0_cloth_0_pants_0_Z1117_XON28_YOP...
3,"[[[85, 87, 92, 93, 84, 90, 89, 87, 87, 87, 90,...",camel,camel_1,camel_1_hair_0_cloth_0_pants_0_Z1120_XON1_YOP0...
4,"[[[18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,...",camel,camel_1,camel_1_hair_0_cloth_0_pants_0_Z1144_XOP5_YOP1...
...,...,...,...,...
19995,"[[[90, 89, 88, 89, 88, 89, 88, 87, 90, 97, 89,...",warrior_II,warrior_II_4,warrior_II_4_hair_3_cloth_3_pants_0_Z940_XOP30...
19996,"[[[18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,...",warrior_II,warrior_II_4,warrior_II_4_hair_3_cloth_3_pants_0_Z958_XON23...
19997,"[[[23, 23, 23, 23, 22, 22, 22, 22, 23, 22, 22,...",warrior_II,warrior_II_4,warrior_II_4_hair_3_cloth_3_pants_0_Z972_XOP6_...
19998,"[[[19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,...",warrior_II,warrior_II_4,warrior_II_4_hair_3_cloth_3_pants_0_Z987_XOP19...


In [13]:
df = read_pickle(v5f_pickle_path)

In [16]:
df["images"].iloc[0][0].shape

(156, 156)

In [17]:
len(df["images"].iloc[0])

5

In [ ]:
# (156, 156, 5)